In [1]:
import pandas as pd
import pickle
from sklearn.feature_selection import VarianceThreshold

In [2]:
df_train = pd.read_pickle('/home/yoshikawa/work/kaggle/OPVP/output/feature_model/20210908/lgbm_stratified_kfold/train.pkl')

In [3]:
df_train.head()

,row_id,target,log_return_realized_volatility,log_return_mean,log_return_std,log_return2_realized_volatility,log_return2_mean,log_return2_std,log_return3_realized_volatility,log_return3_mean,...,trade_log_return_per_amount_realized_volatility_300_relative,trade_log_return_seconds_diff_realized_volatility_300_relative,trade_log_return_realized_volatility_200_relative,trade_log_return_per_size_realized_volatility_200_relative,trade_log_return_per_amount_realized_volatility_200_relative,trade_log_return_seconds_diff_realized_volatility_200_relative,trade_log_return_realized_volatility_100_relative,trade_log_return_per_size_realized_volatility_100_relative,trade_log_return_per_amount_realized_volatility_100_relative,trade_log_return_seconds_diff_realized_volatility_100_relative
0,0-5,0.004136,0.004499,7.613599e-06,0.000260,0.006999,0.000008,0.000404,0.005466,0.000003,...,0.000025,0.000912,-0.000478,-0.000042,-0.000042,0.001752,-0.000578,-0.000025,-0.000025,0.002397
1,0-11,0.001445,0.001204,1.810239e-06,0.000086,0.002476,0.000004,0.000176,0.001782,0.000005,...,-0.000087,-0.007827,-0.000267,-0.000113,-0.000113,-0.008360,-0.000384,-0.000127,-0.000126,-0.010889
2,0-16,0.002168,0.002369,-1.109201e-05,0.000173,0.004801,-0.000008,0.000352,0.002624,-0.000012,...,-0.000112,0.003209,0.000311,0.000179,0.000179,0.004180,0.000454,0.000195,0.000195,0.011861
3,0-31,0.002195,0.002574,-2.376661e-05,0.000236,0.003637,-0.000017,0.000334,0.002356,-0.000013,...,-0.000084,0.030702,-0.000161,-0.000141,-0.000141,0.023069,0.000134,-0.000178,-0.000178,0.033921
4,0-62,0.001747,0.001894,-1.057099e-08,0.000144,0.003257,-0.000002,0.000247,0.002118,0.000001,...,-0.000134,-0.011067,-0.000463,-0.000154,-0.000154,-0.012745,-0.000552,-0.000187,-0.000187,-0.016745


In [4]:
import seaborn as sns

trade_col = [col for col in list(df_train) if 'trade_' in col]
# print(trade_col)
corr = df_train[trade_col].corr(method='pearson')
# ax = sns.heatmap(
#     corr, 
#     vmin=-1,
#     vmax=1,
#     center=0,
#     cmap=sns.diverging_palette(20, 220, n=200),
#     square=True
# )


In [5]:
# corr[corr > 0.9]

In [6]:
feat_corr = set()
for i in range(len(corr.columns)):
    for j in range(i):
        if abs(corr.iloc[i, j]) > 0.9999:
            feat_corr.add(corr.columns[i])
print(len(feat_corr))
print(corr.shape[1])

68
348


In [7]:
print(feat_corr)

{'trade_amount_amax_300', 'trade_log_return_realized_volatility_200_mean_stock', 'trade_amount_amax_200', 'trade_log_return_per_amount_realized_volatility_300_mean_time', 'trade_log_return_per_amount_realized_volatility_200_mean_stock', 'trade_log_return_per_amount_realized_volatility_500_mean_time', 'trade_log_return_per_amount_realized_volatility_300_min_time', 'trade_log_return_per_amount_realized_volatility_200_mean_time', 'trade_amount_mean', 'trade_log_return_per_amount_realized_volatility_500_mean_stock', 'trade_log_return_per_amount_realized_volatility_relative', 'trade_log_return_per_amount_realized_volatility_300', 'trade_log_return_per_amount_realized_volatility', 'trade_amount_sum_400', 'trade_log_return_per_amount_realized_volatility_500_std_stock', 'trade_log_return_per_amount_realized_volatility_400_min_time', 'trade_amount_amin_200', 'trade_amount_amin', 'trade_amount_sum_300', 'trade_log_return_per_amount_realized_volatility_min_stock', 'trade_log_return_per_amount_rea

In [8]:
feat_corr = set()
book_col = [col for col in list(df_train) if col not in trade_col]
book_col.remove('row_id')
book_col.remove('time_id')
book_col.remove('stock_id')
book_col.remove('target')
corr = df_train[book_col].corr(method='pearson')
for i in range(len(corr.columns)):
    for j in range(i):
        if abs(corr.iloc[i, j]) > 0.9999:
            feat_corr.add(corr.columns[i])
print(len(feat_corr))
print(corr.shape[1])

18
550


In [9]:
import numpy as np

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2 
    dfs = []
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    dfs.append(df[col].astype(np.int8))
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    dfs.append(df[col].astype(np.int16))
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    dfs.append(df[col].astype(np.int32))
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    dfs.append(df[col].astype(np.int64) ) 
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    dfs.append(df[col].astype(np.float16))
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    dfs.append(df[col].astype(np.float32))
                else:
                    dfs.append(df[col].astype(np.float64))
        else:
            dfs.append(df[col])
    
    df_out = pd.concat(dfs, axis=1)
    if verbose:
        end_mem = df_out.memory_usage().sum() / 1024**2
        num_reduction = str(100 * (start_mem - end_mem) / start_mem)
        print(f'Mem. usage decreased to {str(end_mem)[:3]}Mb:  {num_reduction[:2]}% reduction')
    return df_out

In [10]:
df_train = reduce_mem_usage(df_train)

Mem. usage decreased to 859Mb:  68% reduction


In [11]:
a = np.array([1.0], dtype=np.float16)
b = np.array([5000000000000])
c = a+b
c.dtype
a = b+c
a.dtype

dtype('float64')